In [2]:
import pandas as pd
file = "purchase_data.csv"
df = pd.read_csv(file)
df.columns

Index(['Purchase ID', 'SN', 'Age', 'Gender', 'Item ID', 'Item Name', 'Price'], dtype='object')

In [10]:
players = df["SN"].value_counts()
players_unique = players.count()
total_players = pd.DataFrame({"Total Number of Players": [players_unique]})
total_players

,Total Number of Players
0,576


In [11]:
items = df["Item ID"].value_counts()
items_unique = items.count()
average_purchase = df["Price"].mean()

number_purchase = df["Purchase ID"].count()
revenue = df["Price"].sum()
pd.options.display.float_format = '${:,.2f}'.format
purchasing_analysis = pd.DataFrame({"Number of Unique Items": [items_unique],
                                   "Average Purchase Price": [average_purchase],
                                    "Total Number of Purchases": [number_purchase],
                                    "Total Revenue": [revenue]
                                   })
purchasing_analysis

,Number of Unique Items,Average Purchase Price,Total Number of Purchases,Total Revenue
0,183,$3.05,780,"$2,379.77"


In [101]:
gender_items = df.loc[:, ["Gender", "SN"]]
gender_items = gender_items.drop_duplicates(["SN"])
count_female_gender = len(gender_items[(gender_items['Gender']== 'Female')])
count_male_gender = len(gender_items[(gender_items['Gender']=='Male')])
count_other_gender = len(gender_items[(gender_items['Gender'] != 'Female') & (gender_items['Gender'] != 'Male')])

percentage_female_gender = count_female_gender / count_total_gender * 100
percentage_male_gender = count_male_gender / count_total_gender * 100
percentage_other_gender = count_other_gender / count_total_gender * 100

gender_demographics = pd.DataFrame({
        'Total Count': {
            'Male': count_male_gender,
            'Female': count_female_gender,
            'Other / Non-Disclosed': count_other_gender
        },
        'Percentage of Players':{
            'Male' : '{:,.2f}%'.format(percentage_male_gender),
            'Female' : '{:,.2f}%'.format(percentage_female_gender),
            'Other / Non-Disclosed' : '{:,.2f}%'.format(percentage_other_gender),
        }
    })
    
gender_demographics

,Total Count,Percentage of Players
Female,81,10.38%
Male,484,62.05%
Other / Non-Disclosed,11,1.41%


In [180]:
purchase_items01 = df.loc[:, ["Gender", "Price"]]
female_table = purchase_items01.loc[purchase_items01["Gender"] == "Female"]
female_count = female_table["Price"].count()
female_average = female_table["Price"].mean()
female_total = female_table["Price"].sum()

male_table = purchase_items01.loc[purchase_items01["Gender"] == "Male"]
male_count = male_table["Price"].count()
male_average = male_table["Price"].mean()
male_total = male_table["Price"].sum()

other_table = purchase_items01[(purchase_items01['Gender'] != 'Female') & (purchase_items01['Gender'] != 'Male')]
other_count = other_table["Price"].count()
other_average = other_table["Price"].mean()
other_total = other_table["Price"].sum()

avg_total_male = male_total / count_male_gender
avg_total_female = female_total / count_female_gender
avg_total_other = other_total / count_other_gender


gender_purchases = pd.DataFrame({
        'Purchase Count': {
            'Male': male_count,
            'Female': female_count,
            'Other / Non-Disclosed': other_count
        },
        'Average Purchase Price':{
            'Male' : male_average,
            'Female' :female_average,
            'Other / Non-Disclosed' : other_average,
        },
        'Total Purchase Value':{
            'Male' : male_total,
            'Female' :female_total,
            'Other / Non-Disclosed' : other_total,
        },
        'Avg Total Purchase per Person':{
            'Male' : avg_total_male,
            'Female' : avg_total_female,
            'Other / Non-Disclosed' : avg_total_other,
        },
    })
    
gender_purchases

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,"$1,967.64",$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


In [198]:
bins = [0, 9, 14, 19, 24, 29, 34, 39, 50]
group_labels = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34",
                "35-39", "40+"]
df["Age Seg"] = pd.cut(df["Age"], bins, labels=group_labels)

age_01 = df.loc[:, ["Age", "SN","Age Seg"]]
age_01 = age_01.drop_duplicates(["SN"])
age_group01 = age_01.groupby("Age Seg")
age_group_count = age_group01[["SN"]].count()
age_group_count = age_group_count.rename(columns={"SN":"Total Count"
                                             })
age_group_porcentaje = age_group_count / players_unique *100
pd.options.display.float_format = '{:,.2f}'.format
table_age_merge00 = pd.merge(age_group_count, age_group_porcentaje, on= "Age Seg")
table_age_merge00 = table_age_merge00.rename(columns={"Total Count_x":"Total Count", "Total Count_y":"Percentage of Players"
                                             })
table_age_merge00

,Total Count,Percentage of Players
Age Seg,,
<10,17,2.95
10-14,22,3.82
15-19,107,18.58
20-24,258,44.79
25-29,77,13.37
30-34,52,9.03
35-39,31,5.38
40+,12,2.08


In [209]:
age_02 = df.loc[:, ["Age", "SN","Age Seg", "Price"]]
age_group02 = age_02.groupby("Age Seg")

age_group_count = age_group02[["Price"]].count()
age_group_count = age_group_count.rename(columns={"Price":"Purchase Count"
                                             })
age_group_average = age_group02[["Price"]].mean()
age_group_average = age_group_average.rename(columns={"Price":"Average Purchase Price"
                                             })
table_age_merge01 = pd.merge(age_group_count, age_group_average, on= "Age Seg")
age_group_total = age_group02[["Price"]].sum()
age_group_total = age_group_total.rename(columns={"Price":"Total Purchase Value"
                                             })

pd.options.display.float_format = '${:,.2f}'.format
table_age_merge02 = pd.merge(table_age_merge01, age_group_total, on= "Age Seg")
table_age_merge02["Avg Total Purchase per Person"] = table_age_merge02["Total Purchase Value"] / table_age_merge00["Total Count"]
table_age_merge02

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Age Seg,,,,
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,"$1,114.06",$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


In [33]:
top_spenders01 = df[["SN", "Item Name"]]
top_clients = top_spenders01.groupby(["SN"])
clients01 = top_clients.count()

top_spenders02 = df[["SN", "Item Name", "Price"]]
top_clients02 = top_spenders02.groupby(["SN"])
clients02 = top_clients02.mean()

top_clients03 = top_spenders02.groupby(["SN"])
clients03 = top_clients03[["Price"]].sum()

table_spenders_merge01 = pd.merge(total_per_cliente, clients02, on= "SN")
table_spenders_merge02 = pd.merge(table_spenders_merge01, clients01, on= "SN")

total_per_cliente = clients03["Price"].max()
total_per_cliente = clients03.sort_values(["Price"], ascending=False)
total_per_cliente.reset_index(inplace=True)

pd.options.display.float_format = '${:,.2f}'.format
table_spenders_merge02 = table_spenders_merge02.rename(columns={"Price_x":"Total Purchase Value", 
                                                                "Price_y":"Average Purchase Price",
                                                                "Item Name":"Total Purchase Value",
                                             })

table_spenders_merge02.head(5)

,SN,Total Purchase Value,Average Purchase Price,Total Purchase Value
0,Lisosia93,$18.96,$3.79,5
1,Idastidru52,$15.45,$3.86,4
2,Chamjask73,$13.83,$4.61,3
3,Iral74,$13.62,$3.40,4
4,Iskadarya95,$13.10,$4.37,3


In [42]:
top_items01 = df[["SN", "Item Name"]]
top_items02 = top_items01.groupby(["Item Name"])
items01 = top_items02.count()

top_items04 = df[["SN", "Item Name", "Price", "Item ID"]]
top_items05 = top_items04.groupby(["Item Name"])
items02 = top_items05[["Price"]].sum()

table_items_merge01 = pd.merge(items01, items02, on= "Item Name")

total_per_item = items02["Price"].max()
total_per_item = items02.sort_values(["Price"], ascending=False)
total_per_item.reset_index(inplace=True)
columns = ["Item ID", "Price"]
table_items_merge01 = table_spenders_merge02.rename(columns={"SN":"Purchase Count", 
                                                            "Price":"Total Purchase Value",
                                             })

table_items_merge01.head(5)

,Purchase Count,Total Purchase Value,Average Purchase Price,Total Purchase Value
0,Lisosia93,$18.96,$3.79,5
1,Idastidru52,$15.45,$3.86,4
2,Chamjask73,$13.83,$4.61,3
3,Iral74,$13.62,$3.40,4
4,Iskadarya95,$13.10,$4.37,3
